# Jedi Council sentiments

![picture](./images/jedi-council.png)

In [39]:
import pandas as pd

In [40]:
yoda = pd.read_csv('inputs/yoda-dialogues.csv')

In [60]:
yoda_filt = yoda[(yoda['character'] != 'narrator') & (yoda['scene']==134)][['character','text']].reset_index()

In [61]:
yoda_filt

,index,character,text
0,28,ANAKIN,A ship...a cup...a speeder.
1,30,YODA,"Good, good, young one. How feel you?"
2,31,ANAKIN,"Cold, sir."
3,32,YODA,Afraid are you?
4,33,ANAKIN,"No, sir."
5,34,MACE WINDU,Afraid to give up your life?
6,35,ANAKIN,I don't think so.
7,37,YODA,"See through you, we can."
8,38,MACE WINDU,Be mindful of your feelings...
9,39,KI-ADI,YOur thoughts dwell on your mother.


In [43]:
import requests

In [44]:
for jedi in list(yoda_filt['character'].unique()):
    requests.get(f'http://localhost:5000/user/create/{jedi}')

In [45]:
data = {"nombre":"Jedi Council, Ep I",
        "usuarios":[11,12,13,14]}
response = requests.get('http://localhost:5000/chat/create', params=data)

In [ ]:
# cambiar a añadir nombres, no ids
# que devuelva el id del chat aquí

In [62]:
userids = pd.DataFrame(list(zip([11,12,13,14], list(yoda_filt['character'].unique()))),columns =['id', 'character'])

In [65]:
result = pd.merge(yoda_filt, userids, on='character').sort_values('index')

In [76]:
for a, b in zip(result['id'], result['text']):
    response = requests.get('http://localhost:5000/chat/1002/addmessage', params={"usuario":a,"mensaje":b})

In [85]:
response = requests.get('http://localhost:5000/chat/1002/sentiment').json()
response

{'compound': -0.9615, 'neg': 0.222, 'neu': 0.703, 'pos': 0.075}

- positive sentiment: compound score >= 0.05
- neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
- negative sentiment: compound score <= -0.05